In [106]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import itertools 
import re
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

In [107]:
import imblearn;

In [108]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samashtishukla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [110]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samashtishukla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [111]:
master_df = pd.read_csv('sample30.csv')

In [112]:
master_df.head(5)

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [113]:
master_df.isna().sum()

id                          0
brand                       0
categories                  0
manufacturer              141
name                        0
reviews_date               46
reviews_didPurchase     14068
reviews_doRecommend      2570
reviews_rating              0
reviews_text                0
reviews_title             190
reviews_userCity        28071
reviews_userProvince    29830
reviews_username           63
user_sentiment              1
dtype: int64

In [114]:
len(master_df.index)

30000

In [115]:
master_df.tail(10)

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
29990,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2016-12-26T00:00:00.000Z,False,True,5,"This whole set of these (mask, shampoo and con...",Smells Amazing!,NaN,NaN,emily646,Positive
29991,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-07T00:00:00.000Z,False,True,5,Took the 48 hour hair challenge using the sham...,Great clay products!,NaN,NaN,meganjoywolfe,Positive
29992,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2016-12-16T00:00:00.000Z,False,True,5,I absolutely love the smell of this product! I...,Smells Amazing,NaN,NaN,kjb2205,Positive
29993,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-23T00:00:00.000Z,False,True,5,I seriously was so surprised after my shower h...,Ends feel so soft!,NaN,NaN,maryyyyyalice,Positive
29994,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-14T00:00:00.000Z,False,True,5,I got to try this conditioner for free and boy...,"By far, my new favorite conditioner",NaN,NaN,smartthunny,Positive
29995,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-23T00:00:00.000Z,False,True,5,I got this conditioner with Influenster to try...,Softness!!,NaN,NaN,laurasnchz,Positive
29996,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-27T00:00:00.000Z,False,True,5,"I love it , I received this for review purpose...",I love it,NaN,NaN,scarlepadilla,Positive
29997,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-21T00:00:00.000Z,False,True,5,First of all I love the smell of this product....,Hair is so smooth after use,NaN,NaN,liviasuexo,Positive
29998,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-11T00:00:00.000Z,False,True,5,I received this through Influenster and will n...,Perfect for my oily hair!,NaN,NaN,ktreed95,Positive
29999,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-19T00:00:00.000Z,False,True,5,I received this product complimentary from inf...,Conditioned into healthy,NaN,NaN,kcoopxoxo,Positive


In [116]:
master_df['reviews_title'] = master_df['reviews_title'].fillna('unknown')

In [117]:
review_df = pd.DataFrame()
Product_review_df = pd.DataFrame()

In [118]:
# concatenating review title and review text
Product_review_df['comb_review'] = master_df['reviews_title'].str.lower() + ' ' + master_df['reviews_text'].str.lower()

In [119]:
Product_review_df['user_sentiment_code'] = master_df['user_sentiment'].str.lower()

In [120]:
Product_review_df['product_name'] = master_df['name'].str.lower()

In [121]:
selected_columns = Product_review_df[["comb_review","user_sentiment_code"]]
review_df = selected_columns.copy()

In [122]:
Product_review_df['user_sentiment_code'].value_counts()

positive    26632
negative     3367
Name: user_sentiment_code, dtype: int64

In [123]:
len(review_df)

30000

In [124]:
Product_review_df.head(5)

,comb_review,user_sentiment_code,product_name
0,just awesome i love this album. it's very good...,positive,pink friday: roman reloaded re-up (w/dvd)
1,good good flavor. this review was collected as...,positive,lundberg organic cinnamon toast rice cakes
2,good good flavor.,positive,lundberg organic cinnamon toast rice cakes
3,disappointed i read through the reviews on her...,negative,k-y love sensuality pleasure gel
4,irritation my husband bought this gel for us. ...,negative,k-y love sensuality pleasure gel


In [125]:
review_df.head(5)

,comb_review,user_sentiment_code
0,just awesome i love this album. it's very good...,positive
1,good good flavor. this review was collected as...,positive
2,good good flavor.,positive
3,disappointed i read through the reviews on her...,negative
4,irritation my husband bought this gel for us. ...,negative


In [126]:
review_df = review_df[review_df['user_sentiment_code'].notnull()]

In [127]:
review_df['user_sentiment_code'].value_counts()

positive    26632
negative     3367
Name: user_sentiment_code, dtype: int64

In [128]:
review_df['user_sentiment_code'] = review_df['user_sentiment_code'].map(lambda x: 1 if x=='positive' else 0)

In [129]:
review_df.head(5)

,comb_review,user_sentiment_code
0,just awesome i love this album. it's very good...,1
1,good good flavor. this review was collected as...,1
2,good good flavor.,1
3,disappointed i read through the reviews on her...,0
4,irritation my husband bought this gel for us. ...,0


In [130]:
len(review_df.index)

29999

In [131]:
review_df.shape

(29999, 2)

In [132]:
review_df.isna().sum()

comb_review            0
user_sentiment_code    0
dtype: int64

In [133]:
## Cleaning Data
def cleanData(text, lowercase = True, remove_stops = False, stemming = True, lemmatization = True):
    txt = str(text)
    
    # Replace apostrophes with standard lexicons
    txt = txt.replace("isn't", "is not")
    txt = txt.replace("aren't", "are not")
    txt = txt.replace("ain't", "am not")
    txt = txt.replace("won't", "will not")
    txt = txt.replace("didn't", "did not")
    txt = txt.replace("shan't", "shall not")
    txt = txt.replace("haven't", "have not")
    txt = txt.replace("hadn't", "had not")
    txt = txt.replace("hasn't", "has not")
    txt = txt.replace("don't", "do not")
    txt = txt.replace("wasn't", "was not")
    txt = txt.replace("weren't", "were not")
    txt = txt.replace("doesn't", "does not")
    txt = txt.replace("'s", " is")
    txt = txt.replace("'re", " are")
    txt = txt.replace("'m", " am")
    txt = txt.replace("'d", " would")
    txt = txt.replace("'ll", " will")
    
    # More cleaning
    txt = re.sub(r"review", "", txt)
    txt = re.sub(r"Review", "", txt)
    txt = re.sub(r"TripAdvisor", "", txt)
    txt = re.sub(r"reviews", "", txt)
    txt = re.sub(r"Hotel", "", txt)
    txt = re.sub(r"what's", "", txt)
    txt = re.sub(r"What's", "", txt)
    txt = re.sub(r"\'s", " ", txt)
    txt = txt.replace("pic", "picture")
    txt = re.sub(r"\'ve", " have ", txt)
    txt = re.sub(r"can't", "cannot ", txt)
    txt = re.sub(r"n't", " not ", txt)
    txt = re.sub(r"I'm", "I am", txt)
    txt = re.sub(r" m ", " am ", txt)
    txt = re.sub(r"\'re", " are ", txt)
    txt = re.sub(r"\'d", " would ", txt)
    txt = re.sub(r"\'ll", " will ", txt)
    txt = re.sub(r" e g ", " eg ", txt)

    
    # Remove punctuation from text
    #txt = ''.join([c for c in text if c not in punctuation])
    txt = txt.replace(".", " ")
    txt = txt.replace(":", " ")
    txt = txt.replace("!", " ")
    txt = txt.replace("&", " ")
    txt = txt.replace("#", " ")
    
    # Remove all symbols
    txt = re.sub(r'[^A-Za-z0-9\s]',r' ',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    txt = re.sub(r'[0-9]',r' ',txt)
    
    # Replace words like sooooooo with so
    txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
    
    # Split attached words
    #txt = " ".join(re.findall('[A-Z][^A-Z]*', txt))   
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    if stemming:
        st = PorterStemmer()
#         print (len(txt.split()))
#         print (txt)
        txt = " ".join([st.stem(w) for w in txt.split()])
    
    if lemmatization:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w, pos='v') for w in txt.split()])

    return txt

In [134]:
testdata = cleanData("My test sooooooo didn't !")
print(testdata)

my test soo do not


In [135]:
# Cleaning the review
review_df['comb_review'] = review_df['comb_review'].map(lambda x: cleanData(x))

In [136]:
len(review_df)

29999

In [137]:
len(master_df)

30000

In [138]:
stopset = set(stopwords.words('english'))

In [139]:
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stopset)

In [140]:
X = vectorizer.fit_transform(review_df.comb_review)
y = review_df.user_sentiment_code

In [141]:
pickle.dump(vectorizer, open('transform.pkl', 'wb'))

In [142]:
X.shape

(29999, 11952)

In [143]:
y.shape

(29999,)

In [144]:
y.head(5)

0    1
1    1
2    1
3    0
4    0
Name: user_sentiment_code, dtype: int64

In [145]:
y.value_counts()

1    26632
0     3367
Name: user_sentiment_code, dtype: int64

In [146]:
review_df.shape

(29999, 2)

In [147]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.30,random_state=42,stratify=y, shuffle=True)

In [148]:
X_train.shape

(20999, 11952)

In [149]:
X_train.shape

(20999, 11952)

In [150]:
X_test.shape

(9000, 11952)

In [151]:
y_test.value_counts()

1    7990
0    1010
Name: user_sentiment_code, dtype: int64

In [152]:
y_train.value_counts()

1    18642
0     2357
Name: user_sentiment_code, dtype: int64

In [153]:
# class imbalance technique
smt = SMOTE()
X_train_sm,y_train_sm = smt.fit_resample(X_train, y_train)

In [154]:
y_train_sm.shape

(37284,)

In [155]:
type(X_train_sm)

scipy.sparse.csr.csr_matrix

In [156]:
X_train_sm.shape

(37284, 11952)

In [157]:
#clf = naive_bayes.BernoulliNB()
clf = naive_bayes.MultinomialNB()

In [158]:
clf.fit(X_train_sm,y_train_sm)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [159]:
accuracy_score(y_train,clf.predict(X_train))

0.8639459021858183

In [160]:
accuracy_score(y_test,clf.predict(X_test))

0.8158888888888889

In [161]:
y_pred_test = clf.predict(X_test)

In [162]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,clf.predict(X_test))

array([[ 608,  402],
       [1255, 6735]])

In [163]:
from sklearn.metrics import f1_score
f1_score(y_test,clf.predict(X_test))

0.890460765518609

### Logistic Regression

In [164]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train_sm, y_train_sm)

/Users/samashtishukla/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [165]:
accuracy_score(y_train,logisticRegr.predict(X_train))

0.8952807276536978

In [166]:
accuracy_score(y_test,logisticRegr.predict(X_test))

0.851

In [167]:
f1_score(y_test,logisticRegr.predict(X_test))

0.9111273112863675

### Random Forest

In [168]:
from sklearn.ensemble import RandomForestClassifier

In [169]:
randomforestRegr = RandomForestClassifier(n_estimators =100,min_samples_split =4)

In [170]:
randomforestRegr.fit(X_train_sm,y_train_sm)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [171]:
accuracy_score(y_train,randomforestRegr.predict(X_train))

1.0

In [172]:
accuracy_score(y_test,randomforestRegr.predict(X_test))

0.8981111111111111

In [173]:
f1_score(y_test,randomforestRegr.predict(X_test))

0.9438147172354636

### XG Boost

In [174]:
import xgboost as xgb

In [175]:
xgbRegr=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
xgbRegr.fit(X_train_sm,y_train_sm)


/Users/samashtishukla/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:55:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [176]:
xgbRegr.score(X_test,y_test)

0.7787777777777778

In [177]:
accuracy_score(y_test,xgbRegr.predict(X_test))

0.7787777777777778

In [178]:
f1_score(y_test,xgbRegr.predict(X_test))

0.8653729122996823

### Choosing Random Forest as best model. Pickling this

In [179]:
filename = 'nlp_model.pkl'
pickle.dump(randomforestRegr, open(filename, 'wb'))

In [181]:
Product_review_df.head(5)

,comb_review,user_sentiment_code,product_name
0,just awesome i love this album. it's very good...,positive,pink friday: roman reloaded re-up (w/dvd)
1,good good flavor. this review was collected as...,positive,lundberg organic cinnamon toast rice cakes
2,good good flavor.,positive,lundberg organic cinnamon toast rice cakes
3,disappointed i read through the reviews on her...,negative,k-y love sensuality pleasure gel
4,irritation my husband bought this gel for us. ...,negative,k-y love sensuality pleasure gel


In [206]:
fetch_comb_review = Product_review_df.loc[Product_review_df['product_name']=="k-y love sensuality pleasure gel"]

In [207]:
fetch_comb_review

,comb_review,user_sentiment_code,product_name
3,disappointed i read through the reviews on her...,negative,k-y love sensuality pleasure gel
4,irritation my husband bought this gel for us. ...,negative,k-y love sensuality pleasure gel
5,not worth it my boyfriend and i bought this to...,negative,k-y love sensuality pleasure gel
6,disappointing bought this earlier today and wa...,negative,k-y love sensuality pleasure gel
7,not happy at all i bought this product for my ...,negative,k-y love sensuality pleasure gel
8,very disappointing my husband and i bought thi...,negative,k-y love sensuality pleasure gel
9,don't buy got as a surprise for my husband the...,positive,k-y love sensuality pleasure gel
10,pretty dissapoitned tried it with my husband a...,negative,k-y love sensuality pleasure gel
11,waste of money purchased this thinking it woul...,positive,k-y love sensuality pleasure gel
12,waste of money bought this to enhance our time...,negative,k-y love sensuality pleasure gel


In [208]:
getFeatureExtract(fetch_comb_review):
    vectorizer = TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stopset)
    X = vectorizer.fit_transform(fetch_comb_review.comb_review)
    return X

SyntaxError: invalid syntax (<ipython-input-208-d9e938fffa52>, line 1)

In [209]:
def getPrediction(product_name):
    
    fetch_comb_review = Product_review_df.loc[Product_review_df['product_name']==product_name]
    X = getFeatureExtract(fetch_comb_review)
    randomforestRegr.predict()
    

SyntaxError: invalid syntax (<ipython-input-209-3123b990fc69>, line 1)

In [210]:
nlpmodel = pickle.load(open("nlp_model.pkl", 'rb'))

# Create new tfidfVectorizer with old vocabulary
#tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True,
 #                         max_features = 500000, vocabulary = tf1.vocabulary_)

In [211]:
vectorizer = pickle.load(open("transform.pkl", 'rb'))


In [247]:
def predict_Sentiment(product_name):
    fetch_comb_review = Product_review_df.loc[Product_review_df['product_name']==product_name]
    print(fetch_comb_review.head())
    movie_vector = vectorizer.transform(fetch_comb_review.comb_review)
    pred = nlpmodel.predict(movie_vector)
    ones = pd.value_counts(pred).filter(items=[1])[1]
    zeros = pd.value_counts(pred).filter(items=[0])[0]
    positive_perc = round(ones/(ones+zeros)*100,2)
    return positive_perc

In [264]:
import numpy as np
testpred = np.array([0,0,0])
ones = pd.value_counts(testpred).filter(items=[1])
zeros = pd.value_counts(testpred).filter(items=[0])
ones
zeros

0    3
dtype: int64

In [265]:
ones

Series([], dtype: int64)

In [266]:
len(ones)

0

In [267]:
len(zeros)

1

In [260]:
if not zeros:
    print("empty")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
type(zeros)

In [ ]:
type(ones)

In [248]:
result  = predict_Sentiment("k-y love sensuality pleasure gel")
result

                                         comb_review user_sentiment_code  \
3  disappointed i read through the reviews on her...            negative   
4  irritation my husband bought this gel for us. ...            negative   
5  not worth it my boyfriend and i bought this to...            negative   
6  disappointing bought this earlier today and wa...            negative   
7  not happy at all i bought this product for my ...            negative   

                       product_name  
3  k-y love sensuality pleasure gel  
4  k-y love sensuality pleasure gel  
5  k-y love sensuality pleasure gel  
6  k-y love sensuality pleasure gel  
7  k-y love sensuality pleasure gel  


81.48

In [213]:
pred

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1])

In [233]:
print(pred)

[1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [214]:
len(pred)

27

In [234]:
pd.value_counts(pred).filter(items=[1])[1]

22

In [240]:
pd.value_counts(pred).filter(items=[0])[0]

5